In [1]:
# Run on GPU server
# import sys
# sys.path.append("../StockPriceForecast/")

In [2]:
import torch
import os
print("Cuda is available : ", torch.cuda.is_available())
print("Cuda num : ", torch.cuda.device_count())  # gpu数量
print("Current cuda index : ", torch.cuda.current_device())
print("Current cuda name : ", torch.cuda.get_device_name(0))


Cuda is available :  True
Cuda num :  1
Current cuda index :  0
Current cuda name :  NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
from libs import *
dataPath = "../StockPriceForecast/dataSet/data/marketData/data.nc"
data_len = DataLength(dataPath)
print("All days : ", data_len)
input_days = 360

predict_days = 5
test_predict_len = 360  # 预测360天
train_dataSet = DataSet(dataPath=dataPath, isel=[1500, data_len - input_days - test_predict_len],
                        targetDays=predict_days,
                        encoderDecoderParameter="close",
                        targetDataParameter="close")
validation_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + 2 * test_predict_len), data_len - test_predict_len],
                             targetDays=predict_days,
                             encoderDecoderParameter="close",
                             targetDataParameter="close")
test_dataSet = DataSet(dataPath=dataPath, isel=[data_len - (input_days + test_predict_len), data_len],
                       targetDays=predict_days,
                       encoderDecoderParameter="close",
                       targetDataParameter="close")
print("Train days : ", train_dataSet.data.shape)
print("Validation days ; ", validation_dataSet.data.shape)
print("Test days : ", test_dataSet.data.shape)

train_dataLoader = DataLoader(train_dataSet)
validation_dataLoader = DataLoader(validation_dataSet)
test_dataLoader = DataLoader(test_dataSet)

print("train_dataLoader : ", len(train_dataLoader))
print("validation_dataLoader : ", len(validation_dataLoader))
print("test_dataLoader : ", len(test_dataLoader))

All days :  2994
Train days :  (774, 1440, 10)
Validation days ;  (720, 1440, 10)
Test days :  (720, 1440, 10)
train_dataLoader :  410
validation_dataLoader :  356
test_dataLoader :  356


In [4]:
from os.path import join
models_store_path = "../StockPriceForecast/Models/"
days_num = len(train_dataLoader)
loss_function_name = "softDTW"
lr = 0.0001
num_epochs = 2
model_name = f"spf_{loss_function_name}_{num_epochs}_{days_num}.pth"
model_store_path = join(models_store_path, model_name)
# train_dataSet = train_dataSet.isel(0, 390 + days_num -1)
# train_dataLoader = DataLoader(train_dataSet)
print(model_store_path)

../StockPriceForecast/Models/spf_softDTW_2_410.pth


In [5]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
model = Seq2SeqAttention(input_size=1440,
                         hidden_size=4096,
                         output_size=1440,
                         num_layers=4,
                         dropout=0.1)
# train_dataLoader.isel(startIndex=950, endIndex=len(train_dataLoader))
model, ls = train_Seq2SeqAttention(model=model,
                                   device=device,
                                   dataLoader=train_dataLoader,
                                   loss_function_name=loss_function_name,
                                   lr=lr,
                                   init_weight=True,
                                   use_scheduler=False,
                                   num_epochs=num_epochs)

epoch 0 training:   0%|          | 0/410 [00:00<?, ?it/s]

enc_outputs :  torch.Size([1, 360, 4096])


TypeError: forward() takes 4 positional arguments but 5 were given

In [ ]:
torch.save(model, model_store_path)



In [ ]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
sys.path.append("../StockPriceForecast/")
# model = torch.load("../StockPriceForecast/Models/spf_softDTW_2_410.pth")
validation_dataLoader.isel(0, 1)
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=5,
                                                    device=device)
predicts = predicts.permute(2, 0, 1).squeeze()
targets = targets.permute(2, 0, 1).squeeze()
print("predicts : ", predicts.shape)
print("targets : ", targets.shape)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
show_index = 0
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts[show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets[show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [ ]:
import numpy as np
print(ls.__len__())
trace = dict(
    # x = np.linspace(0, len(ls), 1),
    y = np.array(ls),
    mode = "lines",
    name = "loss"
)
fig = go.Figure(trace)
fig.show()

In [ ]:
# 继续训练
# Run on GPU server
import sys
sys.path.append("../StockPriceForecast/")
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
from os.path import join
models_store_path = "../StockPriceForecast/Models/"
# train_dataLoader.isel(900, len(train_dataLoader))
days_num = len(train_dataLoader)
loss_function_name = "softDTW"
lr = 0.0001
num_epochs = 4
model_name = f"spf_{loss_function_name}_{num_epochs}_{days_num}.pth"
model_store_path = join(models_store_path, model_name)
print(model_store_path)

In [ ]:
model = torch.load("../StockPriceForecast/Models/spf_softDTW_1_410.pth")

In [ ]:
model, ls = train_Seq2SeqAttention(model=model,
                                   device=device,
                                   dataLoader=train_dataLoader,
                                   loss_function_name=loss_function_name,
                                   init_weight=False,
                                   use_scheduler=False,
                                   lr=lr,
                                   num_epochs=num_epochs)

In [ ]:
model_store_path = "../StockPriceForecast/Models/spf_L1_1_410.pth"
torch.save(model, model_store_path)

In [ ]:
from libs.modules import *
from libs import *
import torch
import torch.nn as nn
import sys
# model = torch.load("../StockPriceForecast/Models/spf_softDTW_1_910.pth")
sys.path.append("../StockPriceForecast/")
validation_dataLoader.isel(0, 100)

In [ ]:
validation_dataLoader.reset()
predicts, targets, attention_weights = eval_Seq2SeqAttention(model=model,
                                                    dataLoader=validation_dataLoader,
                                                    steps=5,
                                                    device=device)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
time_step = 0
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts[0][time_step].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets[0][time_step].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
import numpy as np
show_batch = 6
show_index = 7
predicts_ = predicts[show_batch].permute(1, 0)
targets_ = targets[show_batch].permute(1, 0)
print("predicts : ", predicts.shape)
print("targets : ", targets.shape)
print("predicts_ : ", predicts_.shape)
print("targets_ : ", predicts_.shape)
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts_[show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets_[show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [ ]:
show_index = 2
trace0 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = predicts[0][show_index].numpy(),
    mode = "lines",
    name = "predict"
)
trace1 = go.Scatter(
    # x = np.linspace(0, 1, 30),
    y = targets[0][show_index].numpy(),
    mode = "lines",
    name = "target"
)
data = [trace0, trace1]
py.iplot(data)

In [ ]:
print(ls.__len__())
trace = dict(
    # x = np.linspace(0, len(ls), 1),
    y = np.array(ls),
    mode = "lines",
    name = "loss"
)
fig = go.Figure(trace)
fig.show()

In [ ]:
import torch
a = torch.rand([3, 4, 5])
print(a.shape)
d1, _, _ = a.shape
print(d1)